In [3]:
# Notebook: LTM (TTM) EBITDA & Valuation (improved with quarterly TTM computation)
import yfinance as yf
import pandas as pd
import numpy as np
from IPython.display import display

def compute_ttm_from_quarters(df_quarters, col_name):
    """Sum last 4 quarters for a given column if available. df_quarters expected as DataFrame with quarters as rows (newest first)."""
    if df_quarters is None or df_quarters.empty:
        return None
    if col_name in df_quarters.columns:
        # take up to 4 most recent non-null values
        vals = df_quarters[col_name].dropna().astype(float).head(4)
        if len(vals) == 0:
            return None
        return vals.sum()
    # fallback: try to guess a revenue-like column
    alt = [c for c in df_quarters.columns if 'Revenue' in c or 'total' in c.lower()]
    if alt:
        vals = df_quarters[alt[0]].dropna().astype(float).head(4)
        return vals.sum() if len(vals)>0 else None
    return None

ticker = "ADS.DE"  # change as needed
stock = yf.Ticker(ticker)

# Try to get quarterly financials first (yfinance provides quarterly_financials and financials)
q_income = getattr(stock, 'quarterly_financials', None)
annual_income = getattr(stock, 'financials', None)

# Prepare income DF (newest rows first)
if q_income is not None and not q_income.empty:
    q_income = q_income.T  # rows = quarters
if annual_income is not None and not annual_income.empty:
    annual_income = annual_income.T

# Compute LTM Revenue and LTM EBITDA using quarterly when available, fallback to annual
ltm_revenue = None
ltm_ebitda = None

# Try quarterly TTM for revenue
ltm_revenue = compute_ttm_from_quarters(q_income, 'Total Revenue') if q_income is not None else None
ltm_ebitda = compute_ttm_from_quarters(q_income, 'Ebitda') if q_income is not None else None

# If quarterly TTM not available, fallback to most recent annual reported values
if ltm_revenue is None and annual_income is not None and 'Total Revenue' in annual_income.columns:
    try:
        ltm_revenue = float(annual_income['Total Revenue'].iloc[0])
    except Exception:
        ltm_revenue = None

if ltm_ebitda is None and annual_income is not None:
    if 'Ebitda' in annual_income.columns:
        ltm_ebitda = float(annual_income['Ebitda'].iloc[0])
    else:
        # try compute from Ebit + Depreciation
        ebit = annual_income.get('Ebit', pd.Series([np.nan]*len(annual_income)))
        dep = annual_income.get('Depreciation', pd.Series([0]*len(annual_income)))
        ltm_ebitda = float((ebit.fillna(0) + dep.fillna(0)).iloc[0]) if len(ebit)>0 else None

print('LTM Revenue (TTM if available):', ltm_revenue)
print('LTM EBITDA (TTM if available):', ltm_ebitda)

# Market cap and EV (same approach as earlier)
history = stock.history(period='1d')
last_price = float(history['Close'].iloc[-1]) if not history.empty else None
shares = stock.info.get('sharesOutstanding', None)
market_cap = last_price * shares if (last_price and shares) else None
print('Market Cap:', market_cap)

# Balance sheet for debt/cash
balance = getattr(stock, 'balance_sheet', None)
if balance is not None and not balance.empty:
    balance = balance.T

total_debt = None
cash = None
if balance is not None and not balance.empty:
    for col in ['Total Debt', 'Long Term Debt', 'Short Term Debt', 'Long Term Debt Noncurrent']:
        if col in balance.columns:
            try:
                total_debt = float(balance[col].iloc[0])
                break
            except Exception:
                continue
    for col in ['Cash', 'Cash And Cash Equivalents', 'Cash And Short Term Investments']:
        if col in balance.columns:
            try:
                cash = float(balance[col].iloc[0])
                break
            except Exception:
                continue

net_debt = (total_debt - cash) if (total_debt is not None and cash is not None) else None
enterprise_value = (market_cap + net_debt) if (market_cap is not None and net_debt is not None) else None
print('Enterprise Value:', enterprise_value)

if ltm_ebitda and enterprise_value:
    print('EV/EBITDA:', enterprise_value / ltm_ebitda)

df = pd.DataFrame({
    'Metric': ['LTM Revenue','LTM EBITDA','Enterprise Value'],
    'Value': [ltm_revenue, ltm_ebitda, enterprise_value]
})
df.to_csv('../data/financials_raw.csv', index=False)
display(df)


LTM Revenue (TTM if available): 24700000000.0
LTM EBITDA (TTM if available): 12014000000.0
Market Cap: 27416212393.088562
Enterprise Value: 30873212393.088562
EV/EBITDA: 2.569769634850055


,Metric,Value
0,LTM Revenue,2.470000e+10
1,LTM EBITDA,1.201400e+10
2,Enterprise Value,3.087321e+10
